In [11]:
#import packages
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2
from scrapy.selector import Selector
import os
import spacy
from spacy.en import English
parser = English()
import unicodedata
import random
from time import sleep
import time



%matplotlib inline

In [2]:
# connect to the IMDB api
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [87]:
#get the top 250 movies
top = imdb.top_250()
#put the top 25 movies in a dataframe
top250 = pd.DataFrame(top)
#drop useless columns
top250 = top250.drop('image', axis=1).drop('can_rate',axis=1)
#export to csv
#top250.to_csv('top250.csv',encoding='utf-8')
#store the codes of the top 25 movies in the variable tconst
titleList = [i for i in top250.tconst]

In [14]:
titleList= titleList[:25]

In [15]:
#titlelist2 is a list of movies from 6 genres. Each genre has a range scores from 1 to 10
titleList2 = ['tt1343103','tt2574698','tt5720450','tt0338459','tt4335650','tt2381991','tt2975590','tt1431045','tt0468569','tt0252487','tt4048272','tt2278388','tt0163651','tt3416828','tt2637294','tt0108255','tt1107812','tt0362165','tt4404474','tt0107715',
 'tt0185183',
 'tt0115624',
 'tt0358349',
 'tt0114614',
 'tt0097428',
 'tt0120903',
 'tt0133093',
 'tt5466576','tt2313780',
 'tt0367677',
 'tt0093300',
 'tt3832096',
 'tt1131734',
 'tt4094724',
 'tt0084516',
 'tt0081505',
 'tt4687276','tt0110357', 'tt0126029', 'tt0120855', 'tt0371606', 'tt2017020', 'tt1594972', 'tt0763304', 'tt0330994','tt1623780', 'tt1702443', 'tt1229827', 'tt3797808', 'tt0939684', 'tt1753433', 'tt4382552', 'tt3962848', 'tt5278596',
         'tt2653342', 'tt1754151']

In [16]:
titleList3 = titleList2 + titleList

In [17]:
len(titleList3)

81

In [54]:
title = imdb.get_title_by_id('tt0126029')

user = imdb.get_title_reviews('tt0126029', max_results = 20)


In [67]:
test = user[15]

In [68]:
test.summary

u'Wonderfully irreverent family film with lots of laughs'

In [26]:
test.user_location

u'Colorado'

In [69]:
test.username

u'Kris Morris'

In [85]:
#title.data

In [89]:
test.user_score

3

In [90]:
test.user_score_count

3

In [ ]:
test.

In [92]:
#make a list of reviews with movie title, user review, review score, movie score, and genre
reviewList = []

for i in titleList3:
    title5 = imdb.get_title_by_id(i)
    time.sleep(random.random())
    user2 = imdb.get_title_reviews(i, max_results = 200)
    for user in user2:
        reviewList.append((title5.title, user.user_location, user.summary, user.rating, title5.rating, title5.genres[0]))

In [116]:
df = pd.DataFrame(reviewList)

df.tail(3)

,0,1,2,3,4,5
11038,The Usual Suspects,United States,A film to watch over and over again for the fi...,9.0,8.6,Crime
11039,The Usual Suspects,United States,A movie you need to watch twice to appreciate,9.0,8.6,Crime
11040,The Usual Suspects,None,"""The greatest trick the Devil ever pulled was ...",10.0,8.6,Crime


In [117]:
#turn the list of reviews into a dataframe
df = pd.DataFrame(reviewList, columns = ['title','location','summary_review','review_score','movie_score','genre'])

In [118]:
df.isnull().sum()

title                0
location          1143
summary_review       3
review_score      1330
movie_score          0
genre                0
dtype: int64

In [119]:
#fill na in review score with movie score
df.review_score.fillna(df.movie_score, inplace =True)

In [120]:
#fill na in review score with movie score
df.location.fillna(value = 'UNKNOWN', inplace =True)

In [121]:
# only 3 empty summary reviews, not bad - we can just drop these
df.isnull().sum()

title             0
location          0
summary_review    3
review_score      0
movie_score       0
genre             0
dtype: int64

In [122]:
df = df.dropna()

In [123]:
df.isnull().sum()

title             0
location          0
summary_review    0
review_score      0
movie_score       0
genre             0
dtype: int64

In [128]:
df.head(2)

,title,location,summary_review,review_score,movie_score,genre
0,Raptor Ranch,United States,Really Not Good!,2.0,3.1,Action
1,Raptor Ranch,United Kingdom,You're better off staying well clear from this...,1.0,3.1,Action


In [125]:
def cleaner(string):
    parsed = parser(unicode(string))
    lemmas_unicode = [token.lemma_ for token in parsed]
    lemmas_string = [unicodedata.normalize('NFKD', i).encode('utf-8') for i in lemmas_unicode]
    lstring = ''
    for i in lemmas_string:
        if i not in  ['\n\n', '!','?','.',',','[0-9]+','[)()-\'/]']:
            lstring += i
            lstring += ' '
    return lstring

In [131]:
cleaner(df.summary_review[3])

'of reptile and man '

In [133]:
reviewlist2 = []
for i in df['summary_review']:
    reviewlist2.append(cleaner(i))
reviewlist2[:2]

['really not good ', "you 're better off stay well clear from this ranch "]

In [135]:
df['summary_review'] = reviewlist2

In [136]:
df.tail(2)

,title,location,summary_review,review_score,movie_score,genre
11039,The Usual Suspects,United States,a movie you need to watch twice to appreciate,9.0,8.6,Crime
11040,The Usual Suspects,UNKNOWN,""" the great trick the devil ever pull be convi...",10.0,8.6,Crime


In [138]:
import csv

df.to_csv('summaryMovie.csv', encoding ='utf-8')